# Leren: Programming assignment 5
** This assignment can be done in teams of 2 **

**Student 1:**  <span style="color:red">de Groot</span> (<span style="color:red">10434410</span>)<br>
**Student 2:** <span style="color:red"> Dijkzeul</span> (<span style="color:red">10554386</span>)<br>

-----------------------------------
You may want to use parts of your code from the previous assignment(s) as a starting point for this assignment. 

The code you hand-in should follow the structure from this document. Write down your functions in the cells they belong to. Note that the structure corresponds with the structure from the actual programming assignment. Make sure you read this for the full explanation of what is expected of you. 

**Submission:**

* Make sure your code can be run from top to bottom without errors.
* Include your data files in the zip file.
* Comment your code

One way be sure you code can be run without errors is by quiting iPython completely and then restart iPython and run all cells again (you can do this by going to the menu bar above: Cell > Run all). This way you make sure that no old definitions of functions or values of variables are left (that your program might still be using).

-----------------------------------

If you have any questions ask your teaching assistent. We are here for you.

-----------------------------------

In [4]:
import pandas as pd
import numpy as np
import math

Training = pd.read_csv('digist123-1.csv', sep=";", header=None)
Test = pd.read_csv('digist123-2.csv', sep=";", header=None)

# Make the pandas dataframes numpy arrays
TrainingNpArray = np.array(Training)
TestNpArray = np.array(Test)

Training.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,0,1,11,15,8,0,0,0,0,...,0,0,0,2,15,15,3,0,0,1
1,0,0,4,10,11,4,0,0,0,1,...,0,0,0,5,12,12,12,1,0,1
2,0,0,5,14,12,5,0,0,0,0,...,0,0,0,7,13,16,8,0,0,1
3,0,0,14,10,0,0,0,0,0,0,...,0,0,0,12,16,16,16,16,9,1
4,0,0,5,12,10,4,0,0,0,0,...,0,0,0,8,12,12,4,0,0,1


In [25]:
# Make dict of all the classes
from collections import defaultdict

def makeClassDict(nparray):
    classDict = defaultdict(list) 
    for i in nparray:
        classDict[i[-1]].append(i[:-1])
    return classDict


In [30]:
print np.mean(TrainingNpArray[0])

print np.std(TrainingNpArray[0])



4.70769230769
6.41326288471


In [32]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent